#  SpaceX EDX SQL Server
## Objectives

Writing and executing SQL queries to solve the questions being asked.


<code>As IBM's DB2 guidelines on this notebook doesnt work, I will use SQL Server to replace DB2</code>


#### Display the names of the unique launch sites  in the space mission


In [1]:
import pyodbc
import pandas as pd 
import numpy as np

def dictfetchall(cursor):
    columns = [col[0] for col in cursor.description]
    return [
        dict(zip(columns, row))
        for row in cursor.fetchall()
    ]

conn = pyodbc.connect('Driver={SQL Server};'
                                  'Server=WEI-PC\SQLEXPRESS;'
                                  'Database=SpaceX;'
                                  'User ID=admin;Password=admin;')
cursor = conn.cursor()

cursor.execute("select DISTINCT Launch_Site from SpaceX")
df = pd.DataFrame.from_dict(dictfetchall(cursor))
df

,Launch_Site
0,CCAFS LC-40
1,CCAFS SLC-40
2,KSC LC-39A
3,VAFB SLC-4E


#### Display 5 records where launch sites begin with the string 'KSC'


In [2]:
cursor.execute("select TOP 5 * from SpaceX WHERE Launch_Site LIKE 'KSC%'")
df = pd.DataFrame.from_dict(dictfetchall(cursor))
df

,Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing _Outcome
0,19-02-2017,14:39:00,F9 FT B1031.1,KSC LC-39A,SpaceX CRS-10,2490,LEO (ISS),NASA (CRS),Success,Success (ground pad)
1,16-03-2017,06:00:00,F9 FT B1030,KSC LC-39A,EchoStar 23,5600,GTO,EchoStar,Success,No attempt
2,30-03-2017,22:27:00,F9 FT B1021.2,KSC LC-39A,SES-10,5300,GTO,SES,Success,Success (drone ship)
3,01-05-2017,11:15:00,F9 FT B1032.1,KSC LC-39A,NROL-76,5300,LEO,NRO,Success,Success (ground pad)
4,15-05-2017,23:21:00,F9 FT B1034,KSC LC-39A,Inmarsat-5 F4,6070,GTO,Inmarsat,Success,No attempt


#### Display the total payload mass carried by boosters launched by NASA (CRS)


In [3]:
cursor.execute("select SUM(CAST([PAYLOAD_MASS__KG_] as int)) as TotalPayloadMass from SpaceX where Customer = 'NASA (CRS)'")
df = pd.DataFrame.from_dict(dictfetchall(cursor))
df

,TotalPayloadMass
0,45596


#### Display average payload mass carried by booster version F9 v1.1


In [4]:
cursor.execute("select AVG(CAST([PAYLOAD_MASS__KG_] as int)) as AveragePayloadMass from SpaceX where Booster_Version = 'F9 v1.1'")
df = pd.DataFrame.from_dict(dictfetchall(cursor))
df

,AveragePayloadMass
0,2928


#### List the date where the succesful landing outcome in drone ship was acheived.


In [5]:
cursor.execute("select MIN(CONVERT(date, [Date], 103)) as [First Success] from SpaceX where [Landing _Outcome] = 'Success (drone ship)'")
df = pd.DataFrame.from_dict(dictfetchall(cursor))
df

,First Success
0,2016-04-08


#### List the names of the boosters which have success in ground pad  and have payload mass greater than 4000 but less than 6000


In [6]:
cursor.execute("select Booster_Version from SpaceX where [Landing _Outcome] = 'Success (ground pad)' AND PAYLOAD_MASS__KG_ > 4000 AND PAYLOAD_MASS__KG_ < 6000")
df = pd.DataFrame.from_dict(dictfetchall(cursor))
df

,Booster_Version
0,F9 FT B1032.1
1,F9 B4 B1040.1
2,F9 B4 B1043.1


#### List the total number of successful and failure mission outcomes


In [7]:
cursor.execute("SELECT COUNT(CASE WHEN [Landing _Outcome] like '%Success%' THEN 1 END) AS Success, COUNT(CASE WHEN [Landing _Outcome] not like '%Success%' THEN 1 END) AS Fail FROM SpaceX")
df = pd.DataFrame.from_dict(dictfetchall(cursor))
df

,Success,Fail
0,61,40


#### List the   names of the booster_versions which have carried the maximum payload mass. Use a subquery


In [8]:
cursor.execute("Select Distinct Booster_Version, PAYLOAD_MASS__KG_ from SpaceX where PAYLOAD_MASS__KG_ = (select max(CAST(PAYLOAD_MASS__KG_ as int)) from SpaceX)")
df = pd.DataFrame.from_dict(dictfetchall(cursor))
df

,Booster_Version,PAYLOAD_MASS__KG_
0,F9 B5 B1048.4,15600
1,F9 B5 B1048.5,15600
2,F9 B5 B1049.4,15600
3,F9 B5 B1049.5,15600
4,F9 B5 B1049.7,15600
5,F9 B5 B1051.3,15600
6,F9 B5 B1051.4,15600
7,F9 B5 B1051.6,15600
8,F9 B5 B1056.4,15600
9,F9 B5 B1058.3,15600


#### List the failed landing_outcomes in drone ship, their booster versions, and launch site names for in year 2015


In [9]:
cursor.execute("SELECT Booster_Version, Launch_Site from SpaceX where [Landing _Outcome] like 'Failure (drone ship)' AND Year(CONVERT(date, [Date], 103)) = 2015")
df = pd.DataFrame.from_dict(dictfetchall(cursor))
df

,Booster_Version,Launch_Site
0,F9 v1.1 B1012,CCAFS LC-40
1,F9 v1.1 B1015,CCAFS LC-40


#### Rank the  count of  successful landing_outcomes between the date 2010-06-04 and 2017-03-20 in descending order.


In [10]:
cursor.execute("Select Launch_Site, count(*) from SpaceX where [Landing _Outcome] like '%Success%' AND CONVERT(date, [Date], 103) BETWEEN '2010/06/04' and '2017/03/20' GROUP BY Launch_Site ORDER BY count(*) DESC")
df = pd.DataFrame.from_dict(dictfetchall(cursor))
df

,Launch_Site,
0,CCAFS LC-40,6
1,KSC LC-39A,1
2,VAFB SLC-4E,1


### Author : Ashlin Darius Govindasamy